# LAB4

#### Функция чтения аннотации

In [ ]:
import csv
import typing
import pandas as pd
import cv2 

In [ ]:
def read_file(path: str) -> list[list[str]]:
    '''Читает файл path и возвращает матрицу имён файлов'''
    files: list[list[str]] = []
    csvfile: typing.TextIO = open(path, "r")
    reader: csv.DictReader = csv.DictReader(csvfile, delimiter=",")
    for row in reader:
        files.append([row["full_path"], row["class"]])
    return files

list_file_info: list[list[str]] = read_file("annotation.csv")

#### (1) Создание датафрейма с двумя колонками

In [ ]:
def create_df_with_two_columns(list_files_info):
    df = pd.DataFrame(columns=["Name", "AbsPath"])
    for file_info in list_files_info:
        df.loc[len(df)] = [file_info[1], file_info[0]]
    return df

create_df_with_two_columns(list_file_info)

#### (2,3,4) Именование колонок, метка, форма

In [ ]:
def create_df(list_files_info):
    df = pd.DataFrame(columns=["Name", "AbsPath", "ClassId", "Weight", "Height", "Depth"])
    for file_info in list_file_info:
        class_id = (0 if "cat" == file_info[1] else 1)
        im = cv2.imread(file_info[0])
        h,w,c = im.shape
        df.loc[len(df)] = [file_info[1], file_info[0], class_id, w,h,c]
    return df

df = create_df(list_file_info)
df

#### (5) Статистическая информация

In [ ]:
def get_static_info(df):
    """
    
    """
    return {"Height": df["Height"].mean(), "Weight": df["Weight"].mean(),
            "Depht":  df["Depth"].mean(), "ClassId": df["ClassId"].mean()}

get_static_info(df)

#### (6) Фильтрация по классу

In [ ]:
def df_filter_by_class_id(df, class_id) -> pd.DataFrame:
    return df.loc[ df["ClassId"] == class_id ]

df_filter_by_class_id(df, 1)

#### (7) Фильтрация по размеру и классу

In [ ]:
def df_filter_by_size_and_class_id(df, weight, height, class_id):
     return df.loc[ (df["ClassId"] == class_id) & (df["Weight"] <= weight) & (df["Height"] <= height) ]

df_filter_by_size_and_class_id(df, 320, 320, 1)

#### (8) Добавление поля количества пикселей

In [ ]:
def df_add_pixel_column(df):
    return df.assign(PixelSize = df["Height"] * df["Weight"])

df = df_add_pixel_column(df)
df

In [31]:
def df_group_by_class_id(df):
    return df.groupby(["ClassId"])

group_df = df_group_by_class_id(df)

#### Max

In [32]:
group_df["PixelSize"].max()

ClassId
0    153600
1    153600
Name: PixelSize, dtype: int64

#### Mean

In [33]:
group_df["PixelSize"].mean()

ClassId
0    135480.64
1    138424.64
Name: PixelSize, dtype: float64

#### Min

In [34]:
group_df["PixelSize"].min()

ClassId
0    51200
1    66880
Name: PixelSize, dtype: int64